In [1]:
import os, sys 
%pwd

'/config/workspace/notebooks'

In [2]:
os.chdir("../")
%pwd

'/config/workspace'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    preprocessor_path: Path

In [4]:
from cropPrediction.constants import * 
from cropPrediction.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            preprocessor_path=config.preprocessor_path
        )

        return data_transformation_config

In [6]:
import os
import pandas as pd
import joblib
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from cropPrediction import logger

class DataTransformation:
    def __init__(self, config):
        self.config = config
        self.preprocessor = None 
        self.transformed_df = None

    def get_data_transformation(self):
        try:
            # Load the dataset
            df = pd.read_csv(self.config.data_path)

            # Divide the dataset into independent and dependent features
            X = df.drop(columns=["label"], axis=1)
            y = df["label"]

            logger.info("Dividing the dataset into independent and dependent features completed")

            # Create an instance of LabelEncoder
            label_encoder = LabelEncoder()

            # Fit the label encoder to your categorical labels (y) and transform them
            y_encoded = label_encoder.fit_transform(y)

            logger.info("Encoding Target variable completed")

            numerical_features = X.select_dtypes(exclude="object").columns
            
            # Define the pipeline
            num_pipeline = Pipeline(
                steps=[
                    ("imputer", SimpleImputer(strategy="median")),
                    ("scaler", StandardScaler())])

            # Define the Preprocessor
            preprocessor = ColumnTransformer(
                transformers=[
                    ("num_pipeline", num_pipeline, numerical_features),
                ]
            )

            self.preprocessor = preprocessor

            # Transform the whole data using the preprocessor
            X_transformed = preprocessor.fit_transform(X)

            # Get the updated column names after ordinal encoding
            column_names = numerical_features.to_list()

            # Combine X_transformed and y back into one Dataframe
            self.transformed_df = pd.DataFrame(X_transformed, columns=column_names)
            self.transformed_df["label"] = y_encoded

            logger.info("Data preprocessing completed")

        except Exception as e:
            raise e

    def save_preprocessor(self):
        if self.preprocessor is not None:
            joblib.dump(self.preprocessor, self.config.preprocessor_path)
            logger.info(f"Preprocessor saved to {self.config.preprocessor_path}")
        else:
            logger.warning("Preprocessor is not available. Please call get_data_transformation to create it.") 

    def train_test_split(self, test_size=0.2, random_state=None):
        if self.preprocessor is None:
            raise ValueError("Preprocessor is not available. Please call get_data_transformation.")

        # Split the data into train and test sets
        train, test = train_test_split(self.transformed_df, test_size=test_size, random_state=random_state)

        # Save the encoded train and test sets in the form of CSV files
        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

        logger.info("Split the data into train and test sets.")
        logger.info(f"Shape of train data: {train.shape}")
        logger.info(f"Shape of test data: {test.shape}")

In [7]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.get_data_transformation()
    data_transformation.save_preprocessor()
    data_transformation.train_test_split()
except Exception as e:
    raise e

[2023-09-28 10:44:24,873: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-09-28 10:44:24,875: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-28 10:44:24,878: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-09-28 10:44:24,879: INFO: common: created directory at: artifacts]
[2023-09-28 10:44:24,880: INFO: common: created directory at: artifacts/data_transformation]
[2023-09-28 10:44:24,888: INFO: 3976750005: Dividing the dataset into independent and dependent features completed]
[2023-09-28 10:44:24,890: INFO: 3976750005: Encoding Target variable completed]
[2023-09-28 10:44:24,901: INFO: 3976750005: Data preprocessing completed]
[2023-09-28 10:44:24,905: INFO: 3976750005: Preprocessor saved to artifacts/data_transformation/preprocessor.joblib]
[2023-09-28 10:44:24,938: INFO: 3976750005: Split the data into train and test sets.]
[2023-09-28 10:44:24,939: INFO: 3976750005: Shape of train data: (1760, 8)]
[2023-09-28 10:44:24,940: 